In [1]:
import os

In [46]:
# from pydantic.dataclasses import dataclass
import uuid
from functools import cached_property
from typing import Optional

from loguru import logger

from pydantic import BaseModel

from mangostar import connection
from mangostar.graph_database.graph import CursorAccountant
from mangostar.graph_database.graph import Edge
from mangostar.graph_database.graph import EdgeQuery
from mangostar.graph_database.graph import Node
from mangostar.graph_database.utilz import gen_hex_id
from mangostar.graph_database.utilz import to_snake
from mangostar.settings import ModuleSettings


modset = ModuleSettings()


class GraphController(BaseModel):
    name: str

    def __init__(self, name: Optional[str] = None, **data):
        data["name"] = name or modset.arangoo.graph_name
        data["name"] = to_snake(data["name"])
        super().__init__(**data)

    @property
    def db(self):
        return connection.graph

    @property
    def graph(self):
        if self.db.has_graph(self.name):
            return self.db.graph(self.name)
        else:
            return self.db.create_graph(self.name)



    def __get_collection(self, name: str):
        if self.graph.has_vertex_collection(name):
            return self.graph.vertex_collection(name)
        else:
            return self.graph.create_vertex_collection(name)

    def __get_edge_collection(self, _edge: Edge, is_replace: bool = False):
        if not self.graph.has_edge_definition(_edge.edge_type):
            return self.graph.create_edge_definition(**_edge.to_edge_def())
        if not is_replace:
            return self.graph.edge_collection(_edge.edge_type)
        return self.graph.replace_edge_definition(**_edge.to_edge_def())

    @logger.catch(reraise=True)
    def collection(
        self, name: str = "", edge: Optional[Edge] = None, is_replace: bool = False
    ):
        if edge is not None:
            return self.__get_edge_collection(edge, is_replace=is_replace)
        return self.__get_collection(name)

    def add_node(self, node: Node):
        self.collection(node.kind).insert(node.to_dict())

    def find_one_node(self, kind: str, tags: dict):
        # logger.warning((kind, tags))
        return CursorAccountant(curse=self.collection(kind).find(tags, limit=1))

    def update_match(self, kind: str, tags: dict, vals: dict):
        col = self.collection(kind)
        col.update_match(tags, vals)
        return CursorAccountant(curse=col.find(tags, limit=1))

    @logger.catch(onerror=lambda x: (500, "Very much now"))
    def get_adjacent(self, edge_query: EdgeQuery):
        # Get the edges for a given node. We wrap the query information into an edge class.
        query_node: Node = edge_query.start
        element_container = self.find_one_node(query_node.kind, query_node.record)
        logger.error(element_container)
        element = element_container.element

        # We pass the element id into a string query.
        verticies = self.graph.traverse(start_vertex=element, direction="inbound", vertex_uniqueness="global", edge_uniqueness="global", max_depth=1)
        verticies_only = verticies.get("vertices")
        if len(verticies_only) > 1:
            adjacent_node = verticies_only[-1]
            return adjacent_node
        return element_container

    def add_edge(self, edge_info: Edge):
        edge_col = self.collection(edge=edge_info)
        find_start = self.find_one_node(*edge_info.find_from())
        find_end = self.find_one_node(*edge_info.find_to())
        edge_col.insert(
            {
                "_key": gen_hex_id(),
                "_from": find_start.doc_id,
                "_to": find_end.doc_id,
                **edge_info.props,
            }
        )
        return edge_col


In [3]:
def ghex():
    return uuid.uuid4().hex

# Recreating the test

Given I create two nodes 
I should be able to run a query on the graph. 
Get any node on the edge.


In [4]:
_id = uuid.uuid4().hex
os.environ["ARANOGO_DATABASE"] = f"GDADDYY_{_id[:5]}"
os.environ["ARANOGO_GRAPH_NAME"] = f"GRAPHY_{_id[:5]}"


# Core Enviornment Variables

In [5]:
_g = uuid.uuid4().hex
GRAPH_NAME = f"testing_graph_{_g}"
NODE_KIND = f"test_kind_{_g}"
NODE_BASE_NAME = f"test_node_{_g}"
NODE_EXPLAINER = {
    "searchably": "rekted", 
    "is": "king",
    "identity": _g
}

In [6]:
graph_controller = GraphController()
# node_one = Node(NODE_KIND, ghex(), NODE_EXPLAINER)

# node_two = Node(NODE_KIND, ghex(), NODE_EXPLAINER)

# graph_controller.add_node(node_one)
# graph_controller.add_node(node_two)

# edge_one = Edge(
#     edge_type="test_edge_type",
#     start=node_one,
#     end=node_two,
#     props={"hello": "world"},
# )

# graph_controller.add_edge(edge_one)
# graph_controller.get_adjacent(
#     EdgeQuery(
#         edge_type="test_edge_type",
#         start=node_one,
#         props={"hello": "world"},
#     )
# )
# assert True

In [7]:
db = graph_controller.db
if db.has_graph('school'):
    school = db.graph('school')
else:
    school = db.create_graph('school')

# Rethinking How I'm Testing This Idea Out
The first thing I need to do is ensure I can still follow the main graph database instructions from the tutorial with my currently created database. If I can run a copy and paste job and follow the tutorial perfectly, I'd know if I'm in the right direction. I also should split my steps more to see if there are any params I can change.

In [8]:
if not school.has_edge_definition('teach'):
    teach = school.create_edge_definition(
        edge_collection='teach',
        from_vertex_collections=['teachers'],
        to_vertex_collections=['teachers']
    )

In [9]:
# List edge definitions.
school.edge_definitions()

[{'edge_collection': 'teach',
  'from_vertex_collections': ['teachers'],
  'to_vertex_collections': ['lectures']}]

In [10]:
school.replace_edge_definition(
    edge_collection='teach',
    from_vertex_collections=['teachers'],
    to_vertex_collections=['lectures']
)

<EdgeCollection teach>

In [11]:
school.delete_edge_definition('teach', purge=True)

True

In [12]:
# Create a new vertex collection named "teachers" if it does not exist.
# This returns an API wrapper for "teachers" vertex collection.
if school.has_vertex_collection('teachers'):
    teachers = school.vertex_collection('teachers')
else:
    teachers = school.create_vertex_collection('teachers')

In [13]:
school.vertex_collections()

['lectures', 'teachers']

In [14]:
teachers.properties()

{'id': '4324834',
 'name': 'teachers',
 'system': False,
 'smart': False,
 'type': 2,
 'edge': False,
 'sync': False,
 'status': 3,
 'status_string': 'loaded',
 'global_id': 'c4324834/',
 'cache': False,
 'replication_factor': 3,
 'min_replication_factor': 1,
 'write_concern': 1,
 'shard_count': 1,
 'shard_fields': ['_key'],
 'shard_like': '_graphs',
 'sharding_strategy': 'hash',
 'key_options': {'key_generator': 'traditional', 'user_keys': True}}

In [15]:
teachers.update({'_key': 'jon', 'age': 35})
teachers.replace({'_key': 'jon', 'name': 'Jon', 'age': 36})
teachers.get('jon')
teachers.has('jon')

True

In [16]:
# Get the API wrapper for edge collection "teach".
if school.has_edge_definition('teach'):
    teach = school.edge_collection('teach')
else:
    teach = school.create_edge_definition(
        edge_collection='teach',
        from_vertex_collections=['teachers'],
        to_vertex_collections=['lectures']
    )

In [17]:
if school.has_vertex_collection('lectures'):
    school.vertex_collection('lectures')
else:
    school.create_vertex_collection('lectures')

# The "_id" field is required instead of "_key" field (except for insert).
school.update_vertex({'_id': 'lectures/CSC101', 'difficulty': 'easy'})
school.replace_vertex({'_id': 'lectures/CSC101', 'difficulty': 'hard'})
school.has_vertex('lectures/CSC101')
school.vertex('lectures/CSC101')
# school.delete_vertex('lectures/CSC101')

{'_key': 'CSC101',
 '_id': 'lectures/CSC101',
 '_rev': '_cIYti4q--B',
 'difficulty': 'hard'}

In [18]:
def new_vertex():
    pass

In [19]:
teach.insert({
    '_key': 'jon-CSC101',
    '_from': 'teachers/jon',
    '_to': 'lectures/CSC101'
})

{'_id': 'teach/jon-CSC101', '_key': 'jon-CSC101', '_rev': '_cIYtjJW--A'}

In [20]:
school.update_vertex({'_id': 'lectures/CSC101', 'difficulty': 'easy'})

{'_id': 'lectures/CSC101',
 '_key': 'CSC101',
 '_rev': '_cIYtjO6--_',
 '_old_rev': '_cIYti4q--B'}

In [21]:
school.replace_vertex({'_id': 'lectures/CSC101', 'difficulty': 'hard'})

{'_id': 'lectures/CSC101',
 '_key': 'CSC101',
 '_rev': '_cIYtjTm--_',
 '_old_rev': '_cIYtjO6--_'}

In [22]:
school.has_vertex('lectures/CSC101')

True

In [23]:
school.vertex('lectures/CSC101')

{'_key': 'CSC101',
 '_id': 'lectures/CSC101',
 '_rev': '_cIYtjTm--_',
 'difficulty': 'hard'}

In [24]:
teach.link('teachers/jon', 'lectures/CSC101', data={'online': False})

{'_id': 'teach/8330810', '_key': '8330810', '_rev': '_cIYtjcy--_'}

In [25]:
teach.edges('teachers/jon', direction='in')

{'edges': [], 'stats': {'filtered': 0, 'scanned_index': 0}}

In [26]:
teach.edges('teachers/jon', direction='out')

{'edges': [{'_key': 'jon-CSC101',
   '_id': 'teach/jon-CSC101',
   '_from': 'teachers/jon',
   '_to': 'lectures/CSC101',
   '_rev': '_cIYtjJW--A'},
  {'_key': '8330810',
   '_id': 'teach/8330810',
   '_from': 'teachers/jon',
   '_to': 'lectures/CSC101',
   '_rev': '_cIYtjcy--_',
   'online': False}],
 'stats': {'filtered': 0, 'scanned_index': 2}}

# It's worked so far!!!!!

Why it worked? I'm guessing because the query was actually right. It's easier to start with that assumption compared to the change of replication factor you just made. Now I'm gonna try pushing it a little. I'm gonna create a chain of verticies and see the overall depth of the edges query. Placing a recursive definition first.

In [27]:
def create_link(previous_node:dict=None):
    if previous_node is None:
        previous_node = school.insert_vertex('teachers', {'_key': ghex(), 'hello': f"world-{ghex()}"})
    new_node = school.insert_vertex('teachers', {'_key': ghex(), 'goodbye': f"world-{ghex()}"})
    teach.link(previous_node, new_node, data={'online': False})
    return new_node

In [28]:
if not school.has_edge_definition('teach'):
    teach = school.create_edge_definition(
        edge_collection='teach',
        from_vertex_collections=['teachers'],
        to_vertex_collections=['teachers']
    )

In [29]:
old = None
for _ in range(10):
    old = create_link(old)

In [30]:
teach.edges(old, direction="in").get("edges")

[{'_key': '8330822',
  '_id': 'teach/8330822',
  '_from': 'teachers/5ff301c649334346befe606922e2eff3',
  '_to': 'teachers/db71bca5d61844b7af0a7acab702299c',
  '_rev': '_cIYtlh2--A',
  'online': False}]

In [31]:
old

{'_id': 'teachers/db71bca5d61844b7af0a7acab702299c',
 '_key': 'db71bca5d61844b7af0a7acab702299c',
 '_rev': '_cIYtlfy--_'}

In [32]:
verticies = school.traverse(start_vertex=old, direction="inbound", vertex_uniqueness="global", edge_uniqueness="global", max_depth=1)
verticies_only = verticies.get("vertices")
assert len(verticies_only) > 1
verticies_only[-1]

# Hypothesis Testing

In [35]:
# from random import choice
# from string import printable

# from inflection import underscore, camelize, dasherize

# from faker import Faker
# from hypothesis import given
# from hypothesis import strategies as st
# from hypothesis.strategies import builds
# from hypothesis.strategies import composite

# from mangostar.utils import InsertParameters


# fake_generator = Faker()


In [36]:
# def random_title():
#     available_generator_list = [
#         "color_name",
#         "first_name",
#         "language_name",
#         "last_name",
#         "city",
#         "country",
#         "street_name",
#         "street_suffix",
#         "credit_card_provider",
#     ]
#     selected_attr: str = choice(available_generator_list)

#     return underscore(dasherize(getattr(fake_generator, selected_attr)()))


In [37]:
# random_title()

In [38]:
# @composite
# def complex_nested_dict(draw):
#     title_path = builds(random_title)
#     draw_keys = st.sampled_from(draw(st.lists(title_path, min_size=25)))
#     # # st.sampled_from(draw(draw_keys))
#     # # targets = st.none() | st.booleans() | st.floats() | st.text(printable, min_size=1) | draw_keys
#     # # add_children = lambda children: st.lists(children, max_size=2) | st.dictionaries(
#     # #     draw_keys, children, min_size=1
#     # # )
#     # rec_json = st.recursive(
#     #     st.none() | st.booleans() | st.floats() | st.text(printable, min_size=1) | draw_keys,
#     #     lambda children: st.lists(children, min_size=10) | st.dictionaries(draw_keys, children, min_size=20),
#     # )
#     # sample_one = st.dictionaries(
#     #     st.sampled_from(draw(draw_keys)), targets, min_size=5
#     # )
#     # # sample_one.
#     # sample_two = st.dictionaries(
#     #     st.sampled_from(draw(draw_keys)), targets, min_size=5
#     # )
#     # # recurse_strategy = st.recursive(sample_dict(), sample_dict)
#     # # recurse_strategy = st.recursive(st.booleans(), st.lists)
#     return draw(draw_keys)
#     # return draw(draw_keys)
#     # return draw()
# # .example()
# namy = complex_nested_dict()

In [39]:
# complex_nested_dict().example()

In [40]:
# complex_nested_dict().example()

In [41]:
# @composite
# def extract_nested(draw):
#     json = st.recursive(

#         st.sampled_from([st.booleans(), st.floats() , namy , st.datetimes() , st.ip_addresses(), st.integers()]),

#         lambda children: st.lists(children) | st.dictionaries(namy, draw(children))
#     ).filter(lambda value: isinstance(value, (list, dict)))
#     return json


In [42]:
# extract_nested().example()

In [43]:
main_types = st.one_of(st.booleans(), st.floats() , namy , st.datetimes() , st.ip_addresses())

NameError: name 'st' is not defined

In [ ]:
st.dictionaries(namy, main_types, min_size=5).example()

{'jerry village': False,
 'croatia': False,
 'tunnel': False,
 'ewe': False,
 'parks': False}

In [ ]:
json.example()

[None, None, None, None, None, None, None, None, None, None]